In [1]:
import pandas as pd
import numpy as np


import finaldata
from finaldata import process_region, load_data, split_data
from transform import classify_features, transform_data, fit_pipeline
import os

In [2]:
base_path = os.path.dirname(os.path.abspath(finaldata.__file__))  # Gets the current working directory
lmp_file = os.path.join(base_path, "Final Data", "Raw", "lmp.csv")
reserve_file = os.path.join(base_path, "Final Data","Raw", "reserve.csv")
hvdc_file = os.path.join(base_path, "Final Data", "Raw","hvdc.csv")
weatherpath = os.path.join(base_path,"Final Data","Raw/")

In [ ]:
region_groups = {
    "Luzon": [
        "Cubi Point", "NAIA", "Science Garden", "San Jose", "Tayabas",
        "CLSU", "Tanay", "Ambulong", "Casiguran", "Clark", "Calapan"
    ],
    "Visayas": [
        "Catbalogan", "Roxas City", "Catarman", "Maasin", "Dumaguete"
    ],
    "Mindanao": [
        "Davao City", "Surigao", "Zamboanga", "Dipolog", "Butuan",
        "Malaybalay", "General Santos", "Cotabato"
    ]
}

In [ ]:
date_start = pd.Timestamp('2022-01-01')
min_date = pd.Timestamp('2023-01-01')
date_end = pd.Timestamp('2023-12-31')

In [ ]:
luzon = process_region(lmp_file,hvdc_file,reserve_file,weatherpath,"LUZ", region_groups["Luzon"], date_start,date_end)
visayas = process_region(lmp_file,hvdc_file,reserve_file,weatherpath,"VIS", region_groups["Visayas"], date_start,date_end)
mindanao = process_region(lmp_file,hvdc_file,reserve_file,weatherpath,"MIN", region_groups["Mindanao"], min_date,date_end)

NameError: name 'region_groups' is not defined

In [2]:
luzon=load_data("LUZ")
visayas=load_data("VIS")
mindanao=load_data("MIN")
luzon = luzon.set_index('RUN_TIME')
visayas = visayas.set_index('RUN_TIME')
mindanao = mindanao.set_index('RUN_TIME')

In [3]:
# For LSTM (use_val=True, returns 6 values)
train,val,test = split_data(luzon,  use_val=True)


In [4]:
print(train.shape)


(438, 40)


In [ ]:
def check_transform( data, regionname):
    minmax_cols, boxcox_cols, yeojohnson_cols = classify_features(data)

    # Fit and get pipeline instances
    

    train, val, test = split_data(data,True)

    boxcox_pipeline = fit_pipeline(train, boxcox_cols, regionname, method='box-cox')
    yeojohnson_pipeline = fit_pipeline(train, yeojohnson_cols, regionname, method='yeo-johnson')
    minmax_pipeline = fit_pipeline(train, minmax_cols, regionname, method='minmax')

    
    train_bc = boxcox_pipeline.transform(train[boxcox_cols])
    train_yj = yeojohnson_pipeline.transform(train[yeojohnson_cols])
    train_mm = minmax_pipeline.transform(train[minmax_cols])

    train_transformed = np.hstack([train_mm, train_bc, train_yj])
    return train_transformed

In [14]:
train_transformed= check_transform(luzon,"LUZ")

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- FLOW_LUZ
- RAINFALL_Ambulong
- RAINFALL_CLSU
- RAINFALL_Calapan
- RAINFALL_Casiguran
- ...
